In [3]:
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
import os
os.chdir('..')
import json
import pickle
from datetime import datetime

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import xgboost as xgb

In [ ]:
df = pd.read_parquet("./output/processed_data.parquet")

,timestamp,building_id,electricity_consumption,site_id,building_id_kaggle,site_id_kaggle,primaryspaceusage,sub_primaryspaceusage,sqm,sqft,...,day_of_week_sin,day_of_week_cos,month_sin,month_cos,electricity_lag1,electricity_lag24,electricity_lag168,electricity_rolling_mean_24h,electricity_rolling_std_24h,electricity_rolling_mean_7d
0,2016-01-01 00:00:00,Bear_assembly_Angel,237.85,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,0.00,0.0,0.0,237.85,237.850000,237.85
1,2016-01-01 01:00:00,Bear_assembly_Angel,221.25,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,237.85,0.0,0.0,229.55,11.737973,229.55
2,2016-01-01 02:00:00,Bear_assembly_Angel,222.50,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,221.25,0.0,0.0,227.20,9.244323,227.20
3,2016-01-01 03:00:00,Bear_assembly_Angel,221.40,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,222.50,0.0,0.0,225.75,8.085893,225.75
4,2016-01-01 04:00:00,Bear_assembly_Angel,224.25,Bear,602.0,4.0,Entertainment/public assembly,Entertainment/public assembly,22117.0,238065.0,...,-0.433884,-0.900969,0.5,0.866025,221.40,0.0,0.0,225.45,7.034646,225.45


In [7]:
df.describe()

,timestamp,electricity_consumption,building_id_kaggle,site_id_kaggle,sqm,sqft,lat,lng,yearbuilt,numberoffloors,...,day_of_week_sin,day_of_week_cos,month_sin,month_cos,electricity_lag1,electricity_lag24,electricity_lag168,electricity_rolling_mean_24h,electricity_rolling_std_24h,electricity_rolling_mean_7d
count,25187366,2.518737e+07,2.361971e+07,2.484978e+07,2.518737e+07,2.518737e+07,2.173823e+07,2.173823e+07,2.518737e+07,2.518737e+07,...,2.518737e+07,2.518737e+07,2.518737e+07,2.518737e+07,2.518737e+07,2.518737e+07,2.518737e+07,2.518737e+07,2.518737e+07,2.518737e+07
mean,2017-01-05 02:26:02.344139264,1.415233e+02,7.174411e+02,6.775220e+00,8.385567e+03,9.026153e+04,4.024820e+01,-7.371624e+01,9.518174e+02,1.008576e+00,...,-2.978396e-03,-1.190349e-03,-1.776797e-02,-2.774845e-03,1.416032e+02,1.415948e+02,1.405814e+02,1.418331e+02,2.330233e+01,1.419798e+02
min,2016-01-01 00:00:00,1.000000e-04,0.000000e+00,0.000000e+00,2.630000e+01,2.830000e+02,2.851769e+01,-1.222607e+02,0.000000e+00,0.000000e+00,...,-9.749279e-01,-9.009689e-01,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-04,0.000000e+00,1.000000e-04
25%,2016-07-10 07:00:00,1.934000e+01,3.600000e+02,3.000000e+00,2.043900e+03,2.200000e+04,3.342443e+01,-9.774310e+01,0.000000e+00,0.000000e+00,...,-7.818315e-01,-9.009689e-01,-8.660254e-01,-8.660254e-01,1.933330e+01,1.921000e+01,1.851000e+01,2.049000e+01,2.955675e+00,2.094428e+01
50%,2017-01-06 02:00:00,5.943000e+01,7.070000e+02,5.000000e+00,5.261000e+03,5.662900e+04,3.890350e+01,-7.700535e+01,0.000000e+00,0.000000e+00,...,0.000000e+00,-2.225209e-01,-2.449294e-16,-1.836970e-16,5.942000e+01,5.924000e+01,5.830250e+01,6.159708e+01,9.981142e+00,6.206327e+01
75%,2017-07-05 09:00:00,1.563400e+02,1.074000e+03,1.200000e+01,1.049400e+04,1.129560e+05,4.497878e+01,-7.648529e+01,1.968000e+03,1.000000e+00,...,7.818315e-01,6.234898e-01,5.000000e-01,8.660254e-01,1.563300e+02,1.561000e+02,1.550000e+02,1.552841e+02,2.682654e+01,1.547004e+02
max,2017-12-31 23:00:00,2.626800e+03,1.448000e+03,1.500000e+01,8.129010e+04,8.750000e+05,5.334980e+01,6.260300e+00,2.017000e+03,2.600000e+01,...,9.749279e-01,1.000000e+00,1.000000e+00,1.000000e+00,2.212280e+05,2.212280e+05,2.212280e+05,1.164667e+04,4.495546e+04,5.694944e+03
std,NaN,2.363174e+02,4.129450e+02,4.858215e+00,1.016949e+04,1.094636e+05,7.292841e+00,3.782269e+01,9.830988e+02,2.345526e+00,...,7.073738e-01,7.068325e-01,7.042538e-01,7.097205e-01,2.450145e+02,2.472536e+02,2.477590e+02,2.347898e+02,7.674869e+01,2.337894e+02


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25187366 entries, 0 to 25187365
Data columns (total 61 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   timestamp                     datetime64[ns]
 1   building_id                   object        
 2   electricity_consumption       float64       
 3   site_id                       object        
 4   building_id_kaggle            float64       
 5   site_id_kaggle                float64       
 6   primaryspaceusage             object        
 7   sub_primaryspaceusage         object        
 8   sqm                           float64       
 9   sqft                          float64       
 10  lat                           float64       
 11  lng                           float64       
 12  timezone                      object        
 13  electricity                   object        
 14  hotwater                      object        
 15  chilledwater                  